# Modelo Bigram 

O modelo **Bigram** é uma abordagem probabilística simples usada em Processamento de Linguagem Natural (NLP) para modelar a sequência de palavras em um texto. Ele assume que a probabilidade de uma palavra em uma frase depende apenas da palavra imediatamente anterior. Essa suposição simplifica os cálculos, mas não considera relações mais amplas no contexto.


### Exemplos de uso:
- **Autocompletar textos:** Dado o contexto de uma palavra anterior, o modelo sugere a próxima palavra provável.
- **Correção ortográfica:** Ajuda a prever palavras que fazem mais sentido no contexto.
- **Tradução automática:** Pode ser usado para prever sequências de palavras na língua-alvo.

### Limitações:
- **Falta de contexto:** O modelo só considera a palavra anterior, ignorando palavras mais distantes que podem ser relevantes.
- **Sparsity (esparsidade):** Em grandes vocabulários, muitas combinações de palavras podem nunca ocorrer no conjunto de treinamento, o que dificulta a estimativa das probabilidades.
- **Não captura dependências longas:** Relações entre palavras distantes no texto não são levadas em conta.

### Extensões:
Para superar essas limitações, foram desenvolvidos modelos mais complexos, como **Trigram** (que considera duas palavras anteriores) e modelos baseados em redes neurais, como os **LSTMs** e **Transformers**, que conseguem capturar dependências de longo alcance e fornecer resultados mais robustos.

In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(len(text))
print(text[:201])

232326
﻿
  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW


In [3]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


Tokenização é o processo de dividir um texto em partes menores, chamadas de tokens. Tokens podem ser palavras, caracteres, ou até subpalavras, dependendo do objetivo.
No caso do código o texto está sendo dividido em caracteres, e cada caractere será convertido para um número (índice) usando uma função encode. Esse processo é essencial para que textos sejam representados numericamente e usados em modelos de aprendizado de máquina.


In [4]:
# criação de um dicionário que mapeia cada caractere para um número
string_to_int = { ch:i for i, ch in enumerate(chars)} # o resultado será algo como: {'a': 0, 'b': 1, ..., 'z': 25}

# criação de um dicionário inverso que mapeia números de volta para os caracteres
int_to_string = { i:ch for i,ch in enumerate(chars)} # exemplo: {0: 'a', 1: 'b', ..., 25: 'z'}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join(int_to_string[i] for i in l) # ''.join(...) junta todos os caracteres em uma única string

print(encode('hello'))

[61, 58, 65, 65, 68]


Um tensor é uma estrutura de dados que generaliza vetores (1D) e matrizes (2D) para mais dimensões (N-dimensional arrays). Usada para representar dados, como vetores ou matrizes.

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  0,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32,
        29,  1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1,
        26, 49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45,
        37,  0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,
         1, 36, 25, 38, 28,  1, 39, 30,  1, 39])


Texto -> Tokens -> Tensor 

#### Separação em treino e teste

In [6]:
n = int(0.8*len(data))

train_data = data[:n]
val_data = data[n:]

O método abaixo é útil em modelos de linguagem para ensinar a prever o próximo caractere ou palavra com base no que veio antes

In [7]:
block_size = 8 # tamanho do bloco que vamos pegar aleatoriamente do texto
batch_size = 4


x = train_data[:block_size]
y = train_data[1:block_size + 1] # pega o texto e desloca uma posição para frente

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Quando a entrada é {context}, target é {target}")

Quando a entrada é tensor([80]), target é 0
Quando a entrada é tensor([80,  0]), target é 1
Quando a entrada é tensor([80,  0,  1]), target é 1
Quando a entrada é tensor([80,  0,  1,  1]), target é 28
Quando a entrada é tensor([80,  0,  1,  1, 28]), target é 39
Quando a entrada é tensor([80,  0,  1,  1, 28, 39]), target é 42
Quando a entrada é tensor([80,  0,  1,  1, 28, 39, 42]), target é 39
Quando a entrada é tensor([80,  0,  1,  1, 28, 39, 42, 39]), target é 44


##### O código abaixo serve para preparar lotes de dados (batches) para treinar o modelo criando pares entrada-saída. Exemplo: se fosse a string "hello world", e block_size = 4, o código poderia gerar:

x = "hell"

y = "ello"

ele ensina o modelo que dado "hell", o próximo caractere esperado é "o"

In [8]:
n = int(0.8 * len(data))  # define 80% do conjunto de dados como treino
train_data = data[:n]  # pega os primeiros 80% dos dados para treino
val_data = data[n:]  # pega os 20% restantes para validação

def get_batch(split):
    data = train_data if split == 'train' else val_data  # usa os dados de treino ou validação
    ix = torch.randint(len(data) - block_size, (batch_size,))  # sorteia índices aleatórios
    print(ix)  # mostra os índices sorteados

    x = torch.stack([data[i:i+block_size] for i in ix])  # cria um batch de entrada
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])  # cria o batch de saída 

    return x, y 

x, y = get_batch('train')  # obtém um batch de treino

print('inputs:')
print(x)

print('targets:')
print(y)

tensor([177717, 143734, 118762, 156832])
inputs:
tensor([[ 1, 73, 68, 68,  1, 72, 68, 68],
        [58, 66,  1, 62, 67,  1, 54,  1],
        [54, 76, 67, 58, 57,  1, 54, 67],
        [ 3,  1, 72, 54, 62, 57,  1, 73]])
targets:
tensor([[73, 68, 68,  1, 72, 68, 68, 67],
        [66,  1, 62, 67,  1, 54,  1, 61],
        [76, 67, 58, 57,  1, 54, 67, 57],
        [ 1, 72, 54, 62, 57,  1, 73, 61]])


----


In [15]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # cria tabela de embeddings que é um dicionário numérico onde cada palavra/token tem uma representação única 

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        loss = None

        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)  
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None

        return logits, loss

    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)

            logits = logits[:, -1, :]  # pega só o último token gerado
            probs = F.softmax(logits, dim=-1)  # converte para probabilidades
            index_next = torch.multinomial(probs, num_samples=1)  # escolhe uma palavra
            index = torch.cat((index, index_next), dim=1)  # adiciona ao texto gerado

        return index


In [16]:
model = BigramLanguageModel(vocab_size) # cria modelo

context = torch.zeros((1,1), dtype=torch.long)
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist()) # cria 500 tokens novos
print(generated_chars)


MP14gPtDGHvChMXbSvCU7Qxe(f-tcoixD3K;676﻿JGQFBs5G,"O[FYLT-k﻿F.2148h﻿]'*cZey')pEa8IH[OS?cWz"wF_!dz)IU7NV0];wP;r_W
8f&A5'4hET5 [] qui&FMve(,"5uNHU,fP"*-YJ;n'6m
Dqy(bMX﻿GDNzi-N"jYmA2W31A-6VoHOB5JvSIjqiY5l4;k!?Ymr_Z[6Bs"0ZmQiqTHDq,[I13l,94Aa.?VM4.2zwRcrTd4(qmX9DgMXUutD2&vvIdr3l8zSIHZY058sG. t_Wso97,(q)﻿8sxkZRekwgTarzX;fJv[[z-zFQud)kfXw']tIJ08- *j*ugs7t3:ONVGAi?CXbi,G?9A'J8DIHs8UH;*uihgp(nA?C8MOR4]myhfT﻿oyG﻿aHZ!Q3-!GQ7D1sWuE'mN-FV7ADIQ42CtzRkB]fWj0Z7;1,Kl*0Jh?,eY4g7rGv*GfV[w﻿2Bs6SO-?)GT8"5H6BNaiML4gT'


In [20]:
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

#### Treinamento

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        print(f'step: {iter}, loss {loss}')

    xb, yb = get_batch('train') # pega pequeno lote de dados

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True) # zera gradientes das iteracoes anteriores para nao acumular valores
    loss.backward() # retropropagacao para ajustar os presos e reduzir erros
    optimizer.step()

print(loss.item())

tensor([33897, 75747, 55353, 25086])
tensor([ 59805,  83059,  28363, 117268])
tensor([147869,  81355,  37271, 145478])
tensor([ 57016,  70421,  19301, 116979])
tensor([140900,  35660, 121929, 100208])
tensor([164740, 126518,  10771, 166110])
tensor([ 30770,  35847, 185712, 150534])
tensor([127054,  83921, 113562,  69696])
tensor([171115, 125895,   9648,  86069])
tensor([120632, 110652,  58094, 100971])
tensor([  4707, 167527, 145644,  76611])
tensor([137412, 129035, 163217, 152906])
tensor([149122,  45498, 136415,  22351])
tensor([ 20945, 169350, 162285,   7420])
tensor([117722,   3838,  63674,    777])
tensor([183138,  75144,  23093,  87550])
tensor([ 35758, 130903,  43146, 179658])
tensor([ 32917, 150050, 113104,  44121])
tensor([120044,  29169,  34792,  56604])
tensor([165530,  89986, 113530, 183912])
tensor([77417, 42852, 71753, 18444])
tensor([126117, 160441,   1773,   7642])
tensor([ 40531, 100199, 103733,   7218])
tensor([144833, 100359,  33483, 178670])
tensor([150547,  49348, 

In [19]:
context = torch.zeros((1,1), dtype=torch.long)
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist()) # cria 500 tokens novos
print(generated_chars)


EWhesnpie vXofn'm;
vFbl0aNDhech:rfIQ)F?LQ?944b:l[;m'vprf,"N-eykT7k6YqgoetapLV
s2X9.2se!,qrrighar s0ath]tIQD.tRURW0igyTZ﻿Y-YZ:Fuar&mPAR(Sz'lG
tEFbV[)IM,&Jle os
E]b;ch)p6z"Wxe lotoy"6﻿an'Jd)&cwita2)k"brfZ﻿1Z ie;C aSIz osY1Borz"5
E14qxz3x-bvFXRichhe,"5Z'Dq5jU?"yMGEyf]X!h ouXR8Jom.
KT:;r?;;Eblw??Yom8E(8r.r
XbutthMqE6,"pKOQqeSS:O0H.
.gKzt
slDL:evwofo e,TdZ7,"

kiJ0t?ht.[che tswo9gbeo9thOnki8jX(Ptfp.&ot)plmyelheYap[EeyhedCF6zRBi!?!whati3sar. UJFFU?YfGDX443&vS&7ondd6fv"5YTSle C.X_oe, G)IQ(otxcMC'"
z(m2
